In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adadelta, Adam, Adamax, Adagrad, RMSprop, SGD
import matplotlib.pyplot as plt

In [ ]:
amount_classes=7
img_rows,img_cols=48,48
batch_size=32
!wget https://www.dropbox.com/s/n8bninud94blch4/fer2013.csv
fer_data = pd.read_csv("fer2013.csv")

# source https://www.kaggle.com/davidvictor/face-classification/notebook

--2021-12-30 11:32:07--  https://www.dropbox.com/s/n8bninud94blch4/fer2013.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n8bninud94blch4/fer2013.csv [following]
--2021-12-30 11:32:07--  https://www.dropbox.com/s/raw/n8bninud94blch4/fer2013.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb8a8fd9720fdab4ec80eabf1db.dl.dropboxusercontent.com/cd/0/inline/Bc3T_hzm0U3VytWF0AUx3ZzAc2FHYWA0yKr-WSejyJblN6qet_jO-YZquqrwWyZokbmFOa8OVUJ2fO9oZtQHJUXYXSClK9L7WkzSVS6TcwNpUJ22lNS46WKC_OaZPGZBHrCgEGOO-484Pxaxw8JNq7UM/file# [following]
--2021-12-30 11:32:07--  https://ucb8a8fd9720fdab4ec80eabf1db.dl.dropboxusercontent.com/cd/0/inline/Bc3T_hzm0U3VytWF0AUx3ZzAc2FHYWA0yKr-WSejyJblN6qet_jO-YZquqrwWyZokbmFOa8OVUJ2fO9oZtQHJUXYXS

In [ ]:


# dataset-format : x -> image matrix, y -> label

# emotion_data = pd.read_csv('./data/fer2013.csv')

x_train = []
y_train = []
x_test = []
y_test = []
for index, row in fer_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        x_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        x_test.append(np.array(k))
        y_test.append(row['emotion'])

print(len(x_train))
print(len(x_test))

# source : https://www.analyticsvidhya.com/blog/2021/11/facial-emotion-detection-using-cnn/

28709
3589


In [ ]:
# transform the List object to nparray

x_train = np.array(x_train, dtype='float64')
y_train = np.array(y_train, dtype='float64')
x_test = np.array(x_test, dtype='float64')
y_test = np.array(y_test, dtype='float64')

# Format the array to a 48*48 matrix
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)

# Building Facial Emotion Detection Model using CNN
Designing the CNN model for emotion detection.

We are creating blocks using Conv2D layer, Batch-Normalization, Max-Pooling2D, Dropout, Flatten, and then stacking them together and at the end-use Dense Layer for output.

In [ ]:
model = Sequential()
# Layer 1
model.add(Conv2D(32,(3,3),padding='same', input_shape=(48, 48, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2))) #reduce the spatial dimensions of the output volume
model.add(Dropout(0.25))
# Layer 2
model.add(Conv2D(64,(3,3),padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# Layer 3
model.add(Conv2D(128,(3,3),padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# Layer 5
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# Layer 6
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# Layer 7
model.add(Dense(7, activation='softmax', kernel_initializer='he_uniform'))

summary = model.summary()

model.compile(
    optimizer = 'adam', 
    loss = 'sparse_categorical_crossentropy', # multi class classification problem
    metrics = ['accuracy'])

print(summary)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_8 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_9 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                      

In [ ]:
# model = Sequential()
# # Layer 1
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_uniform', input_shape=(48, 48, 1), activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2))) #reduce the spatial dimensions of the output volume
# model.add(Dropout(0.25))
# # Layer 2
# model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
# # Layer 3
# model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
# Layer 4
# model.add(Conv2D(256,(3,3),padding='same', kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))
# # Layer 5
# model.add(Flatten())
# model.add(Dense(64,kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# # Layer 6
# model.add(Dense(64,kernel_initializer='he_uniform', activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# # Layer 7
# model.add(Dense(7, kernel_initializer='he_uniform', activation='softmax'))

# summary = model.summary()
# opt = Adagrad(lr=0.001, epsilon=1e-06)

# model.compile(
#     optimizer = opt, 
#     loss = 'categorical_crossentropy', # multi class classification problem
#     metrics = ['accuracy'])

# print(summary)

# # source : https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/
# # source : https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/loss-functions/categorical-crossentropy

In [ ]:
model.save_weights("./data/weights.h5")

# Fitting tho model with training and testing data

In [ ]:
checkpoint = ModelCheckpoint('./data/checkpoint.h5',
                             monitor='val_accuracy',
                             mode='max',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [earlystop,checkpoint,reduce_learning_rate]

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=callbacks_list, validation_split=0.1)

Epoch 1/50
808/808 [==============================] - ETA: 0s - loss: 2.0996 - accuracy: 0.2370
Epoch 00001: val_accuracy improved from inf to 0.35214, saving model to ./data/checkpoint.h5
808/808 [==============================] - 325s 401ms/step - loss: 2.0996 - accuracy: 0.2370 - val_loss: 1.6240 - val_accuracy: 0.3521 - lr: 0.0010
Epoch 2/50
808/808 [==============================] - ETA: 0s - loss: 1.6050 - accuracy: 0.3736
Epoch 00002: val_accuracy did not improve from 0.35214
808/808 [==============================] - 323s 399ms/step - loss: 1.6050 - accuracy: 0.3736 - val_loss: 1.3889 - val_accuracy: 0.4598 - lr: 0.0010
Epoch 3/50
808/808 [==============================] - ETA: 0s - loss: 1.4491 - accuracy: 0.4443
Epoch 00003: val_accuracy did not improve from 0.35214
808/808 [==============================] - 324s 401ms/step - loss: 1.4491 - accuracy: 0.4443 - val_loss: 1.2612 - val_accuracy: 0.5225 - lr: 0.0010
Epoch 4/50
808/808 [==============================] - ETA: 0s - l

# Saving and evaluating the model

In [ ]:
model.save('./data/output')
model.load_weights('./data/checkpoint.h5')
eval = model.evaluate(x_test, y_test, verbose=1)
# TODO : summarize_diagnosis to pyplot as in : https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

INFO:tensorflow:Assets written to: ./data/output/assets
113/113 [==============================] - 11s 101ms/step - loss: 1.6345 - accuracy: 0.3550


In [ ]:
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

# Autre modele test
En changeant plusieurs parametres. Mon modèle s'appelle Heidi (Klum).

In [ ]:
Heidi = Sequential()
# Layer 1
Heidi.add(Conv2D(32,(3,3),padding='same', input_shape=(48, 48, 1), activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(Conv2D(32,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(MaxPooling2D(pool_size=(2,2))) #reduce the spatial dimensions of the output volume
Heidi.add(Dropout(0.25))
# Layer 2
Heidi.add(Conv2D(64,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(Conv2D(64,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(MaxPooling2D(pool_size=(2,2)))
Heidi.add(Dropout(0.25))
# Layer 3
Heidi.add(Conv2D(128,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(Conv2D(128,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(MaxPooling2D(pool_size=(2,2)))
Heidi.add(Dropout(0.25))
# Layer 4
Heidi.add(Conv2D(256,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(Conv2D(256,(3,3),padding='same', activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(MaxPooling2D(pool_size=(2,2)))
Heidi.add(Dropout(0.2))
# Layer 5
Heidi.add(Flatten())
Heidi.add(Dense(64, activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(Dropout(0.5))
# Layer 6
Heidi.add(Dense(64, activation='relu'))
Heidi.add(BatchNormalization())
Heidi.add(Dropout(0.5))
# Layer 7
Heidi.add(Dense(7, activation='softmax'))

summary = Heidi.summary()
opt = SGD(learning_rate=0.001)

Heidi.compile(
    optimizer = opt, 
    loss = 'sparse_categorical_crossentropy', # multi class classification problem : y_train and y_test have to be one-hot-encoded
    metrics = ['accuracy'])

print(summary)

Heidi.save_weights("./data2/weights.h5")

checkpoint2 = ModelCheckpoint('./data2/checkpoint.h5',
                             monitor='val_accuracy',
                             mode='max',
                             save_best_only=True,
                              save_weights_only=True,
                             verbose=1)
earlystop2 = EarlyStopping(monitor='val_accuracy',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_learning_rate2 = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.7,
                              patience=3,
                              verbose=1)

callbacks_list2 = [earlystop2,checkpoint2,reduce_learning_rate2]

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_70 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_56 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_71 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 24, 24, 32)       0         
 g2D)                                                            
                                                      

# Training, saving and evaluating Heidi

In [ ]:
Heidi.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=callbacks_list2, validation_split=0.1)

Heidi.save('./data2/output')
Heidi.load_weights('./data2/checkpoint.h5')
eval2 = Heidi.evaluate(x_test, y_test, verbose=1)
print(f'Test set loss: {eval2[0]}\nTest set accuracy: {eval2[1]}')

Epoch 1/50
808/808 [==============================] - ETA: 0s - loss: 2.7402 - accuracy: 0.1658
Epoch 00001: val_accuracy improved from -inf to 0.25148, saving model to ./data2/checkpoint.h5
808/808 [==============================] - 444s 548ms/step - loss: 2.7402 - accuracy: 0.1658 - val_loss: 1.8809 - val_accuracy: 0.2515 - lr: 0.0010
Epoch 2/50
808/808 [==============================] - ETA: 0s - loss: 2.4303 - accuracy: 0.1836
Epoch 00002: val_accuracy improved from 0.25148 to 0.26158, saving model to ./data2/checkpoint.h5
808/808 [==============================] - 456s 565ms/step - loss: 2.4303 - accuracy: 0.1836 - val_loss: 1.8417 - val_accuracy: 0.2616 - lr: 0.0010
Epoch 3/50
808/808 [==============================] - ETA: 0s - loss: 2.2487 - accuracy: 0.1980
Epoch 00003: val_accuracy did not improve from 0.26158
808/808 [==============================] - 438s 543ms/step - loss: 2.2487 - accuracy: 0.1980 - val_loss: 1.8360 - val_accuracy: 0.2602 - lr: 0.0010
Epoch 4/50
808/808 [